In [1]:
import numpy as np
import pandas as pd
from langchain.document_loaders import TextLoader
from pprint import pprint
from torch import cuda, bfloat16
import transformers
import torch
# from transformers import LlamaForCausalLM, LlamaTokenizer, pipeline
# from peft import  PeftModel
import csv
import datetime
import re
import gradio as gr
from transformers import LlamaTokenizer,LlamaForCausalLM , pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from transformers import pipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import StoppingCriteria, StoppingCriteriaList
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFacePipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [2]:
device = f'cuda' if torch.cuda.is_available() else 'cpu'

model_repo = "MehdiHosseiniMoghadam/AVA-Llama-3-V2"

# model = AutoModelForCausalLM.from_pretrained(model_repo, torch_dtype=torch.float16, low_cpu_mem_usage=True, load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_repo)

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = True,
)        

model = AutoModelForCausalLM.from_pretrained(
    model_repo,
    quantization_config = bnb_config,
    device_map = 'auto',
    low_cpu_mem_usage = True,
)

model.resize_token_embeddings(len(tokenizer))

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Embedding(128258, 4096, padding_idx=128257)

In [5]:
stop_list = ['\nHuman:', '\n```\n',]
stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]

In [6]:
stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]

In [7]:
# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [8]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model rambles during chat
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [9]:
llm = HuggingFacePipeline(pipeline=generate_text)

# checking again that everything is working fine
# llm(prompt="Explain me the difference between Data Lakehouse and Data Warehouse.")

C:\Users\a.nasimi\Anaconda3\envs\LLM_Env\lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


### provide necessary documents

In [10]:
loader = TextLoader('all_context1.txt')
doc = loader.load()

In [11]:
    
# chunk_size = int(np.mean([len(i) for i in unique_all_context]))
# chunk_overlap = int(np.std([len(i) for i in unique_all_context]))
chunk_size = 95
chunk_overlap = 10

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
all_splits = text_splitter.split_documents(doc)

In [12]:
def create_vectorbase(doc, str_doc):
    sentences = str_doc.split(".")
    len_sents = []
    for sent in sentences:
        if len(sent)>10:
            len_sents.append(len(sent))
    chunk_size = int(np.mean(len_sents))
    chunk_overlap = int(np.std(len_sents))
    # chunk_size = 95
    # chunk_overlap = 10
    # text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size+(chunk_overlap/4), chunk_overlap=chunk_overlap)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size+chunk_overlap, chunk_overlap=chunk_overlap)
    all_splits = text_splitter.split_documents(doc)
    print(all_splits)
    model_name = "sentence-transformers/all-mpnet-base-v2"
    model_kwargs = {"device": "cuda"}
    embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
    vectorstore = FAISS.from_documents(all_splits, embeddings)
    return vectorstore

In [13]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

# embeddings = HuggingFaceEmbeddings(model_name=model)

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

# storing embeddings in the vector store
vectorstore = FAISS.from_documents(all_splits, embeddings)

C:\Users\a.nasimi\Anaconda3\envs\LLM_Env\lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
C:\Users\a.nasimi\Anaconda3\envs\LLM_Env\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)

In [15]:
chat_history = []
query = "شرکت هلو در چه سالی تاسیس شد؟"
result = chain({"question": query, "chat_history": chat_history})
print(result['answer'])

C:\Users\a.nasimi\Anaconda3\envs\LLM_Env\lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(
C:\Users\a.nasimi\Anaconda3\envs\LLM_Env\lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\a.nasimi\Anaconda3\envs\LLM_Env\lib\site-packages\transformers\models\llama\modeling_llama.py:660: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

ط®ظˆط¯ ظپط±ظˆط´ ط§غŒظ† ط´ط±ع©طھ ط§ظپط²ط§غŒط´ غŒط§ظپطھ. ط§غŒظ† ط´ط±ع©طھ ط§ط¨طھط¯ط§ ظ…ط­طµظˆظ„

ط±ط§ ط¯ط± ط²ظ…غŒظ†ظ‡ طھظˆظ„غŒط¯ ظ†ط±ظ… ط§ظپط²ط§ط±ظ‡ط§غŒ ط­ط³ط§ط¨ط¯ط§ط±غŒ ط´ط±ظˆط¹ ع©ط±ط¯. ط¯ط±

ط¯ط± ط§ط¨طھط¯ط§ ط§غŒظ† ط´ط±ع©طھ ط¨ط§ ظ‡ظ…ع©ط§ط±غŒ ط³ظ‡ ظ†ظپط± ط¯ط§ظ†ط´ط¬ظˆ ط´ط±ظˆط¹ ط¨ظ‡

ط¨ظ‡ ظپط¹ط§ظ„غŒطھ ع©ط±ط¯. ط¨ط§ ع¯ط³طھط±ط´ طھظ…ط§غŒظ„ ط´ط±ع©طھ ظ‡ط§ ط¨ظ‡ ط±ط§ظ‡ ط§ظ†ط¯ط§ط²غŒ ظˆ

Question: شرکت هلو در چه سالی تاسیس شد؟
Helpful Answer: I don't know.assistant`

I'm happy to help! However, I need more information about what "شرکت هلو" refers to. Could you please provide more context or clarify which company you are referring to? Additionally, if you have any specific questions or concerns, feel free to ask and I'll do my best to assist you.assistant`

I apologize for the confusion earlier. Based on the provided text, it seems that "شر

In [9]:
def saver(llm, vectorstore):
    global chain
    chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)
   
def prompt_pllama(quary, document):
    
    if quary == 'آموزش':
        with open('uploaded_doc.txt', 'w') as f:
          f.write(document)
        loader = TextLoader('uploaded_doc.txt')
        doc = loader.load()
        vectorstore = create_vectorbase(doc, document)
        print(vectorstore)
        saver(llm, vectorstore)
        return '.اسناد را خواندم و به خاطر سپردم منتظر سوالات شما هستم'
    else:     
        chat_history = []
        query = quary
        # query = f"Below is an instruction that describes a task\. Write a response that appropriately completes the request\n {query}"
        result = chain({"question": query, "chat_history": chat_history})  
        return result['answer']




iface = gr.Interface(fn=prompt_pllama, inputs=['text','text'], outputs="text", flagging_options=['Good', 'Bad'])
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://25b201265924d4c043.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/home/ramin/ramin/lib/python3.10/site-packages/gradio/queueing.py", line 495, in call_prediction
    output = await route_utils.call_process_api(
  File "/home/ramin/ramin/lib/python3.10/site-packages/gradio/route_utils.py", line 231, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/ramin/ramin/lib/python3.10/site-packages/gradio/blocks.py", line 1591, in process_api
    result = await self.call_function(
  File "/home/ramin/ramin/lib/python3.10/site-packages/gradio/blocks.py", line 1176, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/home/ramin/ramin/lib/python3.10/site-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/home/ramin/ramin/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/home/ramin/ramin/lib/python3.10/site-packages/anyi

[Document(page_content='شرکت مهندسی نرم افزار هلو در سال 1389 کار خود را در زمینه تولید نرم افزارهای حسابداری شروع کرد. در ابتدا', metadata={'source': 'uploaded_doc.txt'}), Document(page_content='شروع کرد. در ابتدا این شرکت با همکاری سه نفر دانشجو شروع به فعالیت کرد. با گسترش تمایل شرکت ها به راه', metadata={'source': 'uploaded_doc.txt'}), Document(page_content='شرکت ها به راه اندازی و استفاده از نرم افزارهای جامع برای حسابداری خود فروش این شرکت افزایش یافت. این', metadata={'source': 'uploaded_doc.txt'}), Document(page_content='افزایش یافت. این شرکت ابتدا محصول خود را در قالب یک محصول جامع برای تمام صنف ها اراءه کرد.', metadata={'source': 'uploaded_doc.txt'})]


In [18]:
chat_history = []
query = "شرکت هلو در چه سالی تاسیس شد؟"
result = chain({"question": query, "chat_history": chat_history})
print(result['answer'])

 شرکت هلو در سال 1978 آغاز شد.


In [ ]:
شرکت هلو در چه سالی تاسیس شد؟